In [25]:
import tensorflow as tf
import numpy as np
import os
import random
from keras.regularizers import l2

tfk = tf.keras
tfkl = tf.keras.layers

In [26]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [27]:
import warnings
import logging

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)

tf.get_logger().setLevel(logging.ERROR)
tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Data import

In [28]:
dataset_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data'
training_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/training'
validation_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/validation'
test_dir = '/Users/nicolacecere/GitHub/Artificial-Neural-Networks-and-Deep-Learning/SplitData/data/test'

In [29]:
# Images are divided into folders, one for each class.
# If the images are organized in such a way, we can exploit the
# ImageDataGenerator to read them from disk.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator(rescale = 1./255)
valid_data_gen = ImageDataGenerator(rescale = 1./255)
test_data_gen = ImageDataGenerator(rescale = 1./255)

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed
                                               )
valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(96,96),
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed)
test_gen = train_data_gen.flow_from_directory(directory=test_dir,
                                              target_size=(96,96),
                                              color_mode='rgb',
                                              classes=None, # can be set to labels
                                              class_mode='categorical',
                                              batch_size=32,
                                              shuffle=False,
                                              seed=seed)

Found 2835 images belonging to 8 classes.
Found 398 images belonging to 8 classes.
Found 495 images belonging to 8 classes.


## Augmentation

In [30]:
tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
    height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
    channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=None,
    preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
)

In [31]:
# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=20,
                                        horizontal_flip= True,
                                        vertical_flip= True,
                                        brightness_range=(0.6,1.4),
                                        zoom_range=0.6,
                                        fill_mode='nearest',
                                        rescale=1./255) # rescale value is multiplied to the image

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=training_dir,
                                                       target_size=(96,96),
                                                       color_mode='rgb',
                                                       classes=None, # can be set to labels
                                                       class_mode='categorical',
                                                       batch_size=32,
                                                       shuffle=True,
                                                       seed=seed)

Found 2835 images belonging to 8 classes.


## Model

In [32]:
input_shape = (96, 96, 3)
epochs = 200

il primo modello è il migliore fin ora -> provare a migliorare con le cose del terzo modello

In [12]:
def build_model(input_shape):

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    conv1 = tfkl.Conv2D(
        filters=32,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(input_layer)
    pool1 = tfkl.MaxPooling2D()(conv1)

    conv2 = tfkl.Conv2D(
        filters=64,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool1)
    pool2 = tfkl.MaxPooling2D()(conv2)

    conv3 = tfkl.Conv2D(
        filters=128,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool2)
    conv4 = tfkl.Conv2D(
        filters=128,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(conv3)
    pool3 = tfkl.MaxPooling2D()(conv4)

    conv5 = tfkl.Conv2D(
        filters=256,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool3)
    pool4 = tfkl.MaxPooling2D()(conv5)

    conv6 = tfkl.Conv2D(
        filters=256,
        kernel_size=3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed)
    )(pool4)
    pool5 = tfkl.GlobalAveragePooling2D(name='gap')(conv6)

    flattening_layer = tfkl.Flatten(name='Flatten')(pool5)
    dropout = tfkl.Dropout(0.2, seed=seed)(flattening_layer)
    classifier_layer = tfkl.Dense(units=256, name='Classifier', kernel_initializer=tfk.initializers.HeUniform(seed), activation='relu')(dropout)
    dropout = tfkl.Dropout(0.2, seed=seed)(classifier_layer)
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='output_layer')(dropout)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [16]:
def build_model(input_shape):
    tf.random.set_seed(seed)

    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='input_layer')

    x = tfkl.Conv2D(
        filters = 64,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv1')(input_layer)
    x = tfkl.MaxPooling2D(name='mp1')(x)

    x = tfkl.Conv2D(
        filters = 128,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'conv2')(x)
    x = tfkl.MaxPooling2D(name='mp2')(x)

    x = tfkl.Conv2D(
        filters = 256,
        kernel_size = 3,
        padding = 'same',
        activation = 'relu',
        kernel_initializer = tfk.initializers.HeUniform(seed),
        name = 'Conv3')(x)
    x = tfkl.GlobalAveragePooling2D(name='gap')(x) #different pooling function -> some advantages like explainability
    x = tfkl.Dropout(0.3, seed=seed, name='gap_dropout')(x)

    x = tfkl.Dense(
        units = 256,
        activation = 'relu',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'classifier')(x)
    x = tfkl.Dropout(0.3, seed=seed, name='classifier_dropout')(x)

    output_layer = tfkl.Dense(
        units = 8,
        activation = 'softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        name = 'output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs = input_layer, outputs = output_layer, name = 'model')

    # Compile the model
    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

    # Return the model
    return model

In [40]:
def build_model(input_shape):


    # Layer Input -------------------------------------------------------
    input_layer = tfkl.Input(shape=input_shape, name='Input')



    # Layer 1 -----------------------------------------------------------
    conv1 = tfkl.Conv2D(
        filters=25,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer = l2(0.01)
    )(input_layer)

    conv1 = tfkl.BatchNormalization()(conv1)

    leaky_relu_layer1 = tfkl.LeakyReLU()(conv1)

    pool1 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(leaky_relu_layer1)



    # Layer 2 -----------------------------------------------------------
    conv2 = tfkl.Conv2D(
        filters=50,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer = l2(0.01)
    )(pool1)

    conv2 = tfkl.BatchNormalization()(conv2)

    leaky_relu_layer2 = tfkl.LeakyReLU()(conv2)

    pool2 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(leaky_relu_layer2)



    # Layer 3 -----------------------------------------------------------
    conv3 = tfkl.Conv2D(
        filters=100,
        kernel_size=(3, 3),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer = l2(0.01)
    )(pool2)

    conv3 = tfkl.BatchNormalization()(conv3)

    leaky_relu_layer3 = tfkl.LeakyReLU()(conv3)

    pool3 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(leaky_relu_layer3)



    # Layer 4 -----------------------------------------------------------
    conv4 = tfkl.Conv2D(
        filters=200,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer = l2(0.01)
    )(pool3)

    conv4 = tfkl.BatchNormalization()(conv4)

    leaky_relu_layer4 = tfkl.LeakyReLU()(conv4)

    pool4 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(leaky_relu_layer4)



    # Layer 5 -----------------------------------------------------------
    conv5 = tfkl.Conv2D(
        filters=300,
        kernel_size=(5, 5),
        strides = (1, 1),
        padding = 'same',
        kernel_initializer = tfk.initializers.GlorotUniform(seed),
        kernel_regularizer = l2(0.01)
    )(pool4)

    conv5 = tfkl.BatchNormalization()(conv5)

    leaky_relu_layer5 = tfkl.LeakyReLU()(conv5)

    pool5 = tfkl.MaxPooling2D(
        pool_size = (2, 2)
    )(leaky_relu_layer5)




    # Global Average Pooling -----------------------------------------------------------
    glob_pooling = tfkl.GlobalAveragePooling2D(name='GlobalPooling')(leaky_relu_layer5)


    # Dense Layer -----------------------------------------------------------
    classifier_layer1 = tfkl.Dense(units=512, name='Classifier1', kernel_initializer=tfk.initializers.GlorotUniform(seed), kernel_regularizer = l2(0.01))(glob_pooling)

    classifier_layer1 = tfkl.BatchNormalization()(classifier_layer1)

    leaky_relu_layer = tfkl.LeakyReLU()(classifier_layer1)

    leaky_relu_layer = tfkl.Dropout(0.3, seed=seed)(leaky_relu_layer)


    # Output Layer -----------------------------------------------------------
    output_layer = tfkl.Dense(units=8, activation='softmax', kernel_initializer=tfk.initializers.GlorotUniform(seed), name='Output')(leaky_relu_layer)


    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    model.compile(
        loss=tfk.losses.CategoricalCrossentropy(),
        optimizer=tfk.optimizers.Adam(),
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    return model

In [41]:
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)

  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'),
                                                     save_weights_only=True, # True to save only weights
                                                     save_best_only=False) # True to save only the best epoch
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)

  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [42]:
# Build model (for data augmentation training)
model = build_model(input_shape)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 96, 96, 3)]       0         
                                                                 
 conv2d_12 (Conv2D)          (None, 96, 96, 25)        700       
                                                                 
 batch_normalization_14 (Bat  (None, 96, 96, 25)       100       
 chNormalization)                                                
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 96, 96, 25)        0         
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 48, 48, 25)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 48, 48, 50)        11300 

In [43]:
# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='CNN_Aug')

# Train the model
history = model.fit(
    x = aug_train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = aug_callbacks,
    batch_size=32
).history

Epoch 1/200
89/89 [==============================] - 36s 386ms/step - loss: 8.5927 - accuracy: 0.2998 - precision_2: 0.3877 - recall_2: 0.1249 - val_loss: 8.2485 - val_accuracy: 0.1608 - val_precision_2: 0.1608 - val_recall_2: 0.1608
Epoch 2/200
89/89 [==============================] - 32s 362ms/step - loss: 4.7843 - accuracy: 0.3351 - precision_2: 0.4125 - recall_2: 0.1464 - val_loss: 6.1621 - val_accuracy: 0.1608 - val_precision_2: 0.1608 - val_recall_2: 0.1608
Epoch 3/200
89/89 [==============================] - 30s 341ms/step - loss: 3.3071 - accuracy: 0.3654 - precision_2: 0.4581 - recall_2: 0.1714 - val_loss: 3.4624 - val_accuracy: 0.1910 - val_precision_2: 0.3000 - val_recall_2: 0.0075
Epoch 4/200
89/89 [==============================] - 33s 365ms/step - loss: 2.6685 - accuracy: 0.3753 - precision_2: 0.4708 - recall_2: 0.1788 - val_loss: 2.7425 - val_accuracy: 0.3065 - val_precision_2: 0.2264 - val_recall_2: 0.0603
Epoch 5/200
89/89 [==============================] - 32s 353ms/s

In [44]:
# Save best epoch model
model.save("data_augmentation_experiments/CNN_Aug_Best")

In [45]:
# Trained with data augmentation
model_aug = tfk.models.load_model("data_augmentation_experiments/CNN_Aug_Best")
model_aug_test_metrics = model_aug.evaluate(test_gen, return_dict=True)

print()
print("Test metrics with data augmentation")
print(model_aug_test_metrics)


16/16 [==============================] - 2s 86ms/step - loss: 2.0864 - accuracy: 0.3919 - precision_2: 0.4265 - recall_2: 0.1758

Test metrics with data augmentation
{'loss': 2.086407423019409, 'accuracy': 0.39191919565200806, 'precision_2': 0.4264705777168274, 'recall_2': 0.17575757205486298}
